In [ ]:
import h5py
import numpy as np
import os
from tqdm import tqdm

from resnet3d import Resnet3DBuilder

import keras
from keras.utils import to_categorical
from keras.optimizers import SGD, Nadam
from keras.callbacks import ModelCheckpoint,TensorBoard,ReduceLROnPlateau
from keras import backend as K

from sklearn.model_selection import train_test_split

# Training Parameters

In [ ]:
input_shape = (3, 256, 256, 3)

class_labels = {0: 'others', 1: 'construction', 2: 'cultivation', 3: 'uncultivation'}
class_labels_inv = {v: k for k,v in class_labels.items()}
num_classes = len(class_labels)

batch_size = 32
nb_epoch = 100

np.random.seed(42)

In [ ]:
model_name = 'resNet34_newdata2_leakyrelu_sgd'
dataset_path ='dataset_3d_icasp.h5'
if not os.path.exists(model_name):
    os.mkdir(model_name)

In [ ]:
def load_dataset(dataset_path):
    with h5py.File(dataset_path,'r') as f:
        dx = f['data_x'][:]
        dy = f['data_y'][:]  
    return dx, to_categorical(dy, num_classes)

def batch_generator(data_X, data_y, batch_size):
    indexes = np.array(range(len(data_y)))
    n = len(indexes)
    while True:
        batch_start = 0
        batch_end = batch_size
        np.random.shuffle(indexes)
        while batch_start < n:
            index = []
            batch_y = []
            y = []
            index = indexes[batch_start:batch_end]
            batch_x = np.array([data_X[i] for i in index])
            batch_y = np.array([data_y[i] for i in index])
            yield batch_x, batch_y
            batch_start += batch_size   
            batch_end += batch_size

# Model

In [ ]:
model = Resnet3DBuilder.build_resnet_34(input_shape=input_shape, num_outputs=num_classes)
model.load_weights(model_name + '/' + model_name+'.h5')

sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
# nadam = Nadam(lr=0.002, beta_1=0.9, beta_2=0.999, epsilon=K.epsilon(), schedule_decay=0.004)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

model.summary()

# Load and split data

In [ ]:
X, y = load_dataset(dataset_path)
print ('Size of dataset:', len(y))
print(X.shape)

In [ ]:
#X_train, y_train = X[:600], y[:600]
#X_val, y_val = X[600:700], y[600:700]
#X_test, y_test = X[700:], y[700:]
#print('Training: {}\tValidation: {}\tTesting: {}'.format(X_train.shape[0], X_val.shape[0], X_test.shape[0]))
X_train, y_train = X[600:], y[600:]
X_val, y_val = X[200:400], y[200:400]
X_test, y_test = X[:200], y[:200]
print('Training: {}\tValidation: {}\tTesting: {}'.format(X_train.shape[0], X_val.shape[0], X_test.shape[0]))

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
# X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2)
# print('Training: {}\tValidation: {}\tTesting: {}'.format(X_train.shape[0], X_val.shape[0], X_test.shape[0]))

In [ ]:
ot=0
cul=0
const=0
uncul=0
for i in range(0,1325):
    if(y[i][0]==1):
        ot=ot+1
    if(y[i][1]==1):
        const=const+1
    if(y[i][2]==1):
        cul=cul+1
    if(y[i][3]==1):
        uncul=uncul+1
    
print(ot)
print(const)
print(cul)
print(uncul)

In [ ]:
assert not np.any(np.isnan(X_train))
assert not np.any(np.isnan(y_train))
assert not np.any(np.isnan(X_val))
assert not np.any(np.isnan(y_val))
assert not np.any(np.isnan(X_test))
assert not np.any(np.isnan(y_test))

In [ ]:
train_gen = batch_generator(X_train, y_train, batch_size)
val_gen = batch_generator(X_val, y_val, batch_size)
test_gen = batch_generator(X_test, y_test, batch_size)

In [ ]:
vx = X[1000:]
vy=y[1000:]

In [ ]:
vx, vy = val_gen.next()
print (vx.shape, vy.shape)
image_input = vx[1, :,:,:]
print (image_input.shape)

In [ ]:
#@### load best weights and make sub model
from keras.models import Model, load_model
model.load_weights (model_name+'/'+model_name+'.h5')
sub_model = Model (inputs = model.input, outputs = model.get_layer('add_20').output)
sub_model.summary()

### compute activation of selected layer
sub_output = sub_model.predict (vx)
print (sub_output.shape)

In [ ]:
vx.shape

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from matplotlib.backends.backend_pdf import PdfPages

feature = sub_output#[1,:,:,:]#model.predict(batch_imagesX)  # feature is activation of intermediate layer

pdf = PdfPages( 'test.pdf' )
gs = gridspec.GridSpec(8, 16, top=1., bottom=0., right=1., left=0., hspace=0.,
        wspace=0.)
i =0

for g in gs:
    ax = plt.subplot(g)
    ax.imshow(feature[10][0,:,:,i])
    ax.set_xticks([])
    ax.set_yticks([])
    ax.set_aspect('auto')
    i+=1

pdf.savefig()
pdf.close()

In [ ]:
plt.imshow(vx[90,0,:,:])

In [ ]:
plt.savefig(vx[90,2,:,:])

import matplotlib.cm as cm
W = model.layers[1].get_weights()
#print(W)
W = np.squeeze(W)
W = np.reshape(W[0], (3, 256, 5,5))
print("W shape : ", W.shape)

plt.figure(figsize=(15, 15))
plt.title('conv1 weights')
nice_imshow(plt.gca(), make_mosaic(W, 4, 5), cmap=cm.binary)

def layer_to_visualize(layer):
    inputs = [K.learning_phase()] + model.inputs

    _convout1_f = K.function(inputs, [layer.output])
    def convout1_f(X):
        # The [0] is to disable the training phase flag
        return _convout1_f([0] + [X])

    convolutions = convout1_f(img_to_visualize)
    convolutions = np.squeeze(convolutions)

    print ('Shape of conv:', convolutions.shape)

    n = convolutions.shape[0]
    n = int(np.ceil(np.sqrt(n)))

    # Visualization of each filter of the layer
    fig = plt.figure(figsize=(12,8))
    for i in range(len(convolutions)):
        ax = fig.add_subplot(n,n,i+1)
        ax.imshow(convolutions[i], cmap='gray')

# Training

In [ ]:
checkpoint = ModelCheckpoint(model_name+'/'+model_name+'.h5', monitor='val_loss', verbose=2, save_best_only=True, mode='auto')
tensorboard = TensorBoard(log_dir=model_name, batch_size=batch_size, write_graph=True, write_images=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.00001)
earlystop = keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=10, verbose=2, mode='auto')
cvslogger = keras.callbacks.CSVLogger(model_name+'/'+model_name+'.csv', separator=',', append=True)

callbacks = [checkpoint, reduce_lr, cvslogger, tensorboard, earlystop]
train_steps = int(len(y_train)//batch_size)
val_steps = int(len(y_test)//batch_size)
history = model.fit_generator(train_gen, train_steps, epochs=nb_epoch, verbose=1, 
                    max_queue_size=2, validation_data=val_gen, 
                    validation_steps=val_steps, shuffle=True,
                    workers=1, use_multiprocessing= False, 
                    initial_epoch=0, callbacks=callbacks)

# Results and evaluation

In [ ]:
model.metrics_names

sgd + relu --> 76
sgd + selu --> 75
sgd + leakyrelu --> 75
nadam + selu --> 73

In [ ]:
model.evaluate_generator(test_gen, steps=len(X_test)//batch_size)

In [ ]:
preds = model.predict(X_test)
preds_bin = (preds == preds.max(axis=1, keepdims=True)).astype(int)
preds.shape

In [ ]:
pres_classes = []
for p in preds.argmax(axis=1):
    pres_classes.append(class_labels[p])

In [ ]:
truth_classes = []
for p in y_test.argmax(axis=1):
    truth_classes.append(class_labels[p])

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('comparison3.csv')
print(df.shape)
df.head()

In [ ]:
df['3d_pred'] = pres_classes
df['truth'] = truth_classes
df.head()

In [ ]:
print('2d accuracy')
(df['2d_pred'] == df['truth']).sum() / float(len(df))

In [ ]:
print('3d accuracy')
(df['3d_pred'] == dss: 5.587f['truth']).sum() / float(len(df))

In [ ]:
df.to_csv('comparison4.csv', index=False, header=True)

In [ ]:
df = pd.read_csv('comparison.csv')
print(df.shape)
df.head()

In [ ]:
df['3d_pred'] = pres_classes
df['truth'] = truth_classes
df.head()

In [ ]:
np.sum(np.all(preds_bin == y_test.astype(int), axis=1)) / float(len(y_test))

In [ ]:
import matplotlib.pyplot as plt
import cv2
%matplotlib inline

In [ ]:
class_labels = {0: 'others', 1: 'construction', 2: 'cultivation', 3: 'uncultivation'}

In [ ]:
for imgnum in tqdm(range(len(preds))):
#     imgnum = 112

    fig = plt.figure(figsize=(15, 20))
    plt.subplot(131)
    plt.imshow(X_test[imgnum, 0][:,:,::-1])
    plt.axis('off')

    plt.subplot(132)
    plt.imshow(X_test[imgnum, 1][:,:,::-1])
    plt.axis('off')

    plt.subplot(133)
    plt.imshow(X_test[imgnum, 2][:,:,::-1])
    plt.axis('off')

    pred_label = class_labels[preds[imgnum].argmax()]
    truth = class_labels[y_test[imgnum].astype(int)[0]]

    probs_msg = ' /\ '.join([c+': '+str(p) for c,p in zip(class_labels.values(), preds[imgnum])])
    preds_msg = 'pred3d: {} /\ pred2d: {}  /\  truth: {}'.format(pred_label, df['2d_pred'].iloc[imgnum], truth)

    fig.text(0.1, 0.62, probs_msg, fontsize=12)
    fig.text(0.1, 0.6, preds_msg, fontsize=12)

    plt.savefig('final_results_comparison/{}.png'.format(imgnum), bbox_inches='tight')
    plt.close()

In [ ]:
from keras.utils.visualize_util import plot  
plot(model, to_file='model.png')  
